# Collecting arxiv abstract information

In this notebook I collect arxiv information from the abstract pages. No links are followed, thus no crawling is needed. 

In [2]:
from  scrapy import Selector
import requests
import time as time
import numpy as np
import pandas as pd


In [17]:
#function to go trough single one 
def get_single_abstract(url,silent=False,nvers=0):
    check=requests.get(url)
    #status exist if 200 or below 400
    if check.status_code==200:
        #result list which is collects
        results=[]
        id_is=url.split('/')[-1]
        #website id
        results.append(id_is)
        #abstract number excluding version
        if id_is.find('v')>-1:
            t=id_is.split('v')
            results.append(t[0])
            #whetehr final 
            results.append("False")
        else:
            results.append(id_is)
            #version number
            nvers=0
            results.append("True")            
        if silent==False:
            print(f"{url} can be accessed")
        html=check.content
        sel=Selector(text=html)
        #abstract selection
        sel2=sel.xpath('//blockquote//text()')
        #extract content
        p2=sel2.extract()
        #get abstract content replace new line with space 
        abstract=p2[2].replace('\n',' ').strip()
        if silent==False:
            print(abstract)
        #get title
        sel3=sel.xpath('//*[@class="title mathjax"]//text()')
        p3=sel3.extract()
        #get title content replace new line with space 
        title=p3[1].replace('\n',' ').strip()
        results.append(title)
        results.append(abstract)        
        if silent==False:
            print(title)
        #select authors
        sel4=sel.xpath('//*[@class="authors"]//text()')
        p4=sel4.extract()
        #get elements which are not authors or comma or empty elements
        authors=[]
        for x in p4:
            if x!="Authors:" and x!=', ':
                y=x.strip()
                if y!='':
                    authors.append(x.strip())
        results.append(len(authors))   
        results.append(authors)          
        if silent==False:
            print(len(authors),authors)
        #get comments
        sel5=sel.xpath('//*[@class="tablecell comments mathjax"]//text()')
        #extract them 
        if len(sel5)==1:
            comments=sel5.extract()[0].strip()
        else:
            comments=None
        if silent==False:
            #print them 
            print(comments)
        results.append(comments)              
        #subject primary
        sel6=sel.xpath('//*[@class="primary-subject"]//text()')
        prim_subject=sel6.extract()[0]
        results.append(prim_subject)          
        if silent==False:
            #print them 
            print(prim_subject)
        #all subjects (doubles previous  to be figured with no other how to combine them )
        sel7=sel.xpath('//*[@class="tablecell subjects"]//text()')
        p7=sel7.extract()
        #exclude first is new line second is primary gotten above 
        if len(p7)==3:
            subjects=p7[2].replace(";","").replace("'","")
            #split them 
            s_subjects_prel=subjects.split(")")
            #adds back ) and excludes last one 
            n_subjects=len(s_subjects_prel)
            s_subjects=[]
            for i in range(len(s_subjects_prel)-1):
                s_subjects.append(s_subjects_prel[i]+")")
            if silent==False:    
                print(s_subjects)
        else:
            s_subjects=None
            n_subjects=0
        results.append(n_subjects)
        results.append(s_subjects)           
        #should be split by ; when not empty
        if silent==False:
            print(n_subjects,s_subjects)   
        #to add submission time? and size?    
        #getting version string if not export version 
        if url.find('export')>-1:
            sel8=sel.xpath('//*[@class="submission-history"]//b/text()')
        else:
            sel8=sel.xpath('//*[@class="submission-history"]//strong/text()')
        sel8b=sel.xpath('//*[@class="submission-history"]/text()')
        #version number
        vlast=sel8.extract()[0].strip('[').strip(']').strip('v')
        if silent==False:
            print(vlast)
        #version properties
        vprop=sel8b.extract()
        vsel=vprop[-2-2*nvers].strip()
        if silent==False:
            print(vsel)
        #append vesrion number number
        results.append(int(vlast))  
        #append version properties
        results.append(vsel)
        return results, 0
    else:
        print(f"{url} cannot be accessed, status is {check.status_code}")
        return None, 1

In [18]:
#some special (big author list) retracted first
urls=['https://export.arxiv.org/abs/1703.00052','https://export.arxiv.org/abs/2010.10965','https://export.arxiv.org/abs/2010.10966','https://export.arxiv.org/abs/2010.10967']
#make them automatic , 404 when obviously not extsing
#403 access denied when to many thus now xport used 

In [19]:
#the long author list cause problems not clear whether needed to solve 
start_time=time.time()
for url in urls:
    # list of strings
    res,check=get_single_abstract(url)
stop_time=time.time()
print(f"{len(urls)} sites needed {np.round(stop_time-start_time,3)} seconds")
#not vsisible authors are visisble here but also not numbers 

https://export.arxiv.org/abs/1703.00052 cannot be accessed, status is 403
https://export.arxiv.org/abs/2010.10965 cannot be accessed, status is 403
https://export.arxiv.org/abs/2010.10966 cannot be accessed, status is 403
https://export.arxiv.org/abs/2010.10967 cannot be accessed, status is 403
4 sites needed 0.727 seconds


In [5]:
print(res)

['2010.10967', '2010.10967', 'True', 'Safe Handover in Mixed-Initiative Control for Cyber-Physical Systems', 'For mixed-initiative control between cyber-physical systems (CPS) and its users, it is still an open question how machines can safely hand over control to humans. In this work, we propose a concept to provide technological support that uses formal methods from AI -- description logic (DL) and automated planning -- to predict more reliably when a hand-over is necessary, and to increase the advance notice for handovers by planning ahead of runtime. We combine this with methods from human-computer interaction (HCI) and natural language generation (NLG) to develop solutions for safe and smooth handovers and provide an example autonomous driving scenario. A study design is proposed with the assessment of qualitative feedback, cognitive load and trust in automation.', 19, ['Frederik Wiehr', ',', 'Anke Hirsch', ',', 'Florian Daiber', ',', 'Antonio Kruger', ',', 'Alisa Kovtunova', ',',

Looks now good enough at least for standard author lists. Now I build a function which gets all these for all urls starting with one and getting also the previous versions and the following numbers, until some break condition is reached. 

In [15]:
# input starturl, any no v abstratc works, it does not carry over the next month, max_number of primary, number of hours run
#whether get_single_abstract is silent

def get_many(starturl,max_number=20,max_h=1,silent=True):
    start_time=time.time()
    counter=0
    # list of strings
    print(starturl)
    res,check=get_single_abstract(starturl,silent=silent)
    if check==0:
        df2=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T
        counter+=1
    check_time=time.time()
    #as long as time o.k. and iteration o.k. and scrap worked
    while (check_time-start_time)/3600<max_h and counter<max_number and check==0:
        print(counter)
        new_url=starturl[:-5]+str(int(starturl[-5:])+counter).zfill(5)
        print(new_url)
        res,check=get_single_abstract(new_url,silent=silent,nvers=vcounter+1)
        check_time=time.time()
        if check==0:
            df=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T
            df2=pd.concat([df2,df],axis=0,ignore_index=True)
            counter+=1          
            vcounter=0
            #looping over versions
            print(f" version number is {int(df.loc[0,['version_number']])}")
            if int(df.loc[0,['version_number']])>1:
                while int(df.loc[0,['version_number']])>1:            
                    urlb=new_url+'v'+str(int(df.loc[0,['version_number']]-1))
                    print(urlb)
                    res,check=get_single_abstract(urlb,silent=silent,nvers=vcounter+1)
                    df=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T        
                    print(f" version number is {int(df.loc[0,['version_number']])}")
                    df2=pd.concat([df2,df],axis=0,ignore_index=True)       
    return df2

In [16]:
df3=get_many('https://export.arxiv.org/abs/2006.00001',max_number=10,max_h=10,silent=False)
df3.to_csv('abstract_2006_part1.csv', sep=',')


https://export.arxiv.org/abs/2006.00001
https://export.arxiv.org/abs/2006.00001 cannot be accessed, status is 403


UnboundLocalError: local variable 'df2' referenced before assignment

Looking into some content. 

In [4]:
#Combiinng several files. 
df1=pd.read_csv("abstract_2001_part1.csv")
df2=pd.read_csv("abstract_2001_part2.csv")
df3=pd.read_csv("abstract_2001_part3.csv")
df4=pd.read_csv("abstract_2002_part1.csv")
df5=pd.read_csv("abstract_2003_part1.csv")
df6=pd.read_csv("abstract_2004_part1.csv")
df7=pd.read_csv("abstract_2005_part1.csv")
df8=pd.read_csv("abstract_2005_part2.csv")
dfall=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8],axis=0,ignore_index=True)
print(dfall.shape,dfall.loc[:,['Id','Abstract-number','Final','version_number']],dfall.version_number.describe())

(109240, 14)                   Id  Abstract-number  Final  version_number
0         2001.00001       2001.00001   True               2
1         2001.00002       2001.00002   True               1
2         2001.00003       2001.00003   True               3
3       2001.00003v2       2001.00003  False               2
4       2001.00003v1       2001.00003  False               1
...              ...              ...    ...             ...
109235    2005.10229       2005.10229   True               1
109236    2005.10230       2005.10230   True               2
109237  2005.10230v1       2005.10230  False               1
109238    2005.10231       2005.10231   True               1
109239    2005.10232       2005.10232   True               1

[109240 rows x 4 columns] count    109240.000000
mean          1.663100
std           1.255962
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          44.000000
Name: version_number, dtype: float64


Select number of final version abstracts. 

In [5]:
dfs=dfall[(dfall.Final==True)]
print(dfs.shape)

(64614, 14)


More than 54000 different abstract papers. Looking on the main categories. 

In [6]:
print(dfs.Prim_subject.value_counts()[0:59])

Computer Vision and Pattern Recognition (cs.CV)               3620
Machine Learning (cs.LG)                                      3445
Quantum Physics (quant-ph)                                    2230
Computation and Language (cs.CL)                              1931
High Energy Physics - Phenomenology (hep-ph)                  1625
Analysis of PDEs (math.AP)                                    1523
Materials Science (cond-mat.mtrl-sci)                         1399
Mesoscale and Nanoscale Physics (cond-mat.mes-hall)           1397
Astrophysics of Galaxies (astro-ph.GA)                        1353
High Energy Physics - Theory (hep-th)                         1323
General Relativity and Quantum Cosmology (gr-qc)              1229
Combinatorics (math.CO)                                       1209
Signal Processing (eess.SP)                                   1197
Optimization and Control (math.OC)                            1179
High Energy Astrophysical Phenomena (astro-ph.HE)             

Looking on all version of the one with most versions. 

In [17]:
print(dfall[(dfall.version_number==dfall.version_number.max())])
print(dfall[(dfall['Abstract-number']==2002.04395)])

       Unnamed: 0          Id  Abstract-number  Final  \
27350        7548  2002.04395       2002.04395   True   

                                                   Title  \
27350  A lower bound for the modulus of the Dirichlet...   

                                                Abstract  Author_number  \
27350  The present manuscript aims to derive an expre...              1   

                Authors  Comments                   Prim_subject  \
27350  ['Yuri Heymann']  14 pages  General Mathematics (math.GM)   

       n_sec_subjects sec_subjects  version_number  \
27350               0          NaN              44   

                             version_properties  
27350  Sun, 13 Aug 2023 03:51:41 GMT  (434kb,D)  
       Unnamed: 0             Id  Abstract-number  Final  \
27350        7548     2002.04395       2002.04395   True   
27351        7549  2002.04395v43       2002.04395  False   
27352        7550  2002.04395v42       2002.04395  False   
27353        7551  2002.043

All the same, means there is a problem with getting past version information. Is now corrected, but cannot be down loaded. 

In [9]:
#that should now flexibility that it works now 
url='https://export.arxiv.org/abs/2006.00010'
check=requests.get(url)
html=check.content
sel=Selector(text=html)
sel8=sel.xpath('//*[@class="submission-history"]//b/text()')
sel8b=sel.xpath('//*[@class="submission-history"]/text()')
vlast=sel8.extract()[0].strip('[').strip(']').strip('v')
print(vlast)
#version properties
print(sel8b)
vprop=sel8b.extract()
print(vprop)
vsel=vprop[-2].strip()
print(vsel)
print(vprop[-4].strip())

2
[<Selector query='//*[@class="submission-history"]/text()' data='\n'>, <Selector query='//*[@class="submission-history"]/text()' data='\nFrom: Dhanesh Krishnarao ['>, <Selector query='//*[@class="submission-history"]/text()' data=']\n'>, <Selector query='//*[@class="submission-history"]/text()' data='\n'>, <Selector query='//*[@class="submission-history"]/text()' data=' Fri, 29 May 2020 18:00:03 GMT  (2746...'>, <Selector query='//*[@class="submission-history"]/text()' data='\n'>, <Selector query='//*[@class="submission-history"]/text()' data=' Tue, 4 Aug 2020 22:00:06 GMT  (2352k...'>, <Selector query='//*[@class="submission-history"]/text()' data='\n'>]
['\n', '\nFrom: Dhanesh Krishnarao [', ']\n', '\n', ' Fri, 29 May 2020 18:00:03 GMT  (2746kb,D)', '\n', ' Tue, 4 Aug 2020 22:00:06 GMT  (2352kb,D)', '\n']
Tue, 4 Aug 2020 22:00:06 GMT  (2352kb,D)
Fri, 29 May 2020 18:00:03 GMT  (2746kb,D)
